In [1]:

import os 
import glob
import numpy as np
import yaml
import traceback

from utilsOpenSim import runScaleTool, getScaleTimeRange, runIKTool, generateVisualizerJson


In [2]:
trace_file_path = '/home/pfirouzabadi/projects/Hand_Detection/trace_files/m002/m002_trial0_RTMPoseHand5_TopDown_smoothed.trc'
model_path = 'ARM_Hand_Wrist_Model.osim'
setup_file_name = '27scales.xml'
IK_file = '27IK.xml'
root_dir = './Models'
locked_joints = ['r_x','r_y','r_z']

In [3]:
os.path.isfile(trace_file_path)


True

In [4]:
def scale_model(root_dir, trace_file_path, model_path, setup_file_name, locked_joints = ['r_x','r_y','r_z']):

    openSimPipelineDir = root_dir       
    
    openSimFolderName = 'Data'
    
    openSimDir = os.path.join(openSimPipelineDir, openSimFolderName)        
    outputScaledModelDir = os.path.join(openSimDir, 'Scaled_Model')


    os.makedirs(openSimDir, exist_ok=True)
    os.makedirs(outputScaledModelDir, exist_ok=True)
    # Path setup file.
    #CHANGEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
    genericSetupFile4ScalingName = (setup_file_name)
    pathGenericSetupFile4Scaling = os.path.join(
        openSimPipelineDir, genericSetupFile4ScalingName)
    # Path model file.
    pathGenericModel4Scaling = os.path.join(
        openSimPipelineDir, 
        model_path)            
    # Path TRC file.
    ##CHANGEEEEEEEEEEEEEEEEEEEEEEEEEEEE
    pathTRCFile4Scaling =  trace_file_path
    # Get time range.
    try:
        thresholdPosition = 0.003
        maxThreshold = 0.015
        increment = 0.001
        success = False
        while thresholdPosition <= maxThreshold and not success:
            try:
                timeRange4Scaling = getScaleTimeRange(
                    pathTRCFile4Scaling,
                    thresholdPosition=thresholdPosition,
                    thresholdTime=0.1, removeRoot=True)
                success = True
            except Exception as e:
                print(f"Attempt with thresholdPosition {thresholdPosition} failed: {e}")
                thresholdPosition += increment  # Increase the threshold for the next iteration

        # Run scale tool.
        print('Running Scaling')
        pathScaledModel = runScaleTool(
            pathGenericSetupFile4Scaling, 
            pathGenericModel4Scaling,
            5,
            # sessionMetadata['mass_kg'], 
            pathTRCFile4Scaling, 
            timeRange4Scaling, 
            outputScaledModelDir,
            locked_joints = ['r_x','r_y','r_z']
            # subjectHeight=sessionMetadata['height_m'],
            )
    except Exception as e:
        if len(e.args) == 2: # specific exception
            raise Exception(e.args[0], e.args[1])
        elif len(e.args) == 1: # generic exception
            exception = "Musculoskeletal model scaling failed. Verify your setup and try again. Visit https://www.opencap.ai/best-pratices to learn more about data collection and https://www.opencap.ai/troubleshooting for potential causes for a failed neutral pose."
            raise Exception(exception, traceback.format_exc())

    return pathScaledModel


In [5]:
pathScaledModel = scale_model(root_dir, trace_file_path, model_path, setup_file_name)

Static phase of 0.58s detected in staticPose between [4.26, 4.85].
Running Scaling
r_x  -> Coordinate Locked
r_y  -> Coordinate Locked
r_z  -> Coordinate Locked
thorax
clavicle
clavphant
scapula
scapphant
humphant
humphant1
humerus
ulna
radius
proximal_row
capitate
trapezium
trapezoid
hamate
firstmc1
firstmc
proximal_thumb
distal_thumb
secondmc
thirdmc
fourthmc
fifthmc
2proxph
2midph
2distph
3proxph
3midph
3distph
4proxph
4midph
4distph
5proxph
5midph
5distph


In [6]:
def IK_model( root_dir, trace_file_path, scaled_model_path, IK_file):
    trialName = trace_file_path.split('/')[-1]
    openSimPipelineDir = root_dir
    openSimFolderName = 'Data'
    
    openSimDir = os.path.join(openSimPipelineDir, openSimFolderName)   
    pathScaledModel = scaled_model_path
    pathOutputIK = pathScaledModel[:-5]+'.mot'     
    
    # Inverse kinematics.
    outputIKDir = os.path.join(openSimDir, 'Kinematics')
    os.makedirs(outputIKDir, exist_ok=True)
    # Check if there is a scaled model.
    # pathScaledModel = os.path.join(outputScaledModelDir, 
    #                                 sessionMetadata['openSimModel'] + 
    #                                 "_scaled.osim")
    if os.path.exists(pathScaledModel):
        # Path setup file.
        genericSetupFile4IKName = IK_file
        pathGenericSetupFile4IK = os.path.join(
            openSimPipelineDir, genericSetupFile4IKName)
        # Path TRC file.
        #CHANGE
        pathTRCFile4IK = trace_file_path
        # Run IK tool. 
        print('Running Inverse Kinematics')
        try:
            pathOutputIK = runIKTool(
                pathGenericSetupFile4IK, pathScaledModel, 
                pathTRCFile4IK, outputIKDir)
        except Exception as e:
            if len(e.args) == 2: # specific exception
                raise Exception(e.args[0], e.args[1])
            elif len(e.args) == 1: # generic exception
                exception = "Inverse kinematics failed. Verify your setup and try again. Visit https://www.opencap.ai/best-pratices to learn more about data collection and https://www.opencap.ai/troubleshooting for potential causes for a failed trial."
                raise Exception(exception, traceback.format_exc())
    else:
        raise ValueError("No scaled model available.")

    # Write body transforms to json for visualization.
    outputJsonVisDir = os.path.join(outputIKDir,'VisualizerJsons',
                                    trialName)
    os.makedirs(outputJsonVisDir,exist_ok=True)
    outputJsonVisPath = os.path.join(outputJsonVisDir,
                                        trialName + '.json')
    generateVisualizerJson(pathScaledModel, pathOutputIK,
                            outputJsonVisPath, 
                            vertical_offset=0)  
    return pathOutputIK

In [7]:
IK_model( root_dir, trace_file_path, pathScaledModel, IK_file)

Running Inverse Kinematics


[info] Preparing to run InverseKinematicsTool.
[info] Loaded model ARM_Hand_Wrist_Model_scaled from file ./Models/Data/Scaled_Model/ARM_Hand_Wrist_Model_scaled.osim
[warning] Couldn't find file 'thorax.vtp'.
[warning] Couldn't find file 'clavicle.vtp'.
[warning] Couldn't find file 'scapula.vtp'.
[warning] Couldn't find file 'humerus.vtp'.
[warning] Couldn't find file 'ulna.vtp'.
[warning] Couldn't find file 'radius.vtp'.
[warning] Couldn't find file 'lunate.vtp'.
[warning] Couldn't find file 'sdfastSCAPHOIDw.vtp'.
[warning] Couldn't find file 'sdfastPISIFORMw.vtp'.
[warning] Couldn't find file 'sdfastTRIQUETRALw.vtp'.
[warning] Couldn't find file 'capitate.vtp'.
[warning] Couldn't find file 'trapezium.vtp'.
[warning] Couldn't find file 'trapezoid.vtp'.
[warning] Couldn't find file 'hamate.vtp'.
[warning] Couldn't find file '1mc.vtp'.
[warning] Couldn't find file 'thumbprox.vtp'.
[warning] Couldn't find file 'thumbdist.vtp'.
[warning] Couldn't find file '2mc.vtp'.
[warning] Couldn't fin

RuntimeError: std::exception in 'OpenSim::TimeSeriesTable_< double >::TimeSeriesTable_(std::string const &)': File './Models/Data/Kinematics/m002_trial0_RTMPoseHand5_TopDown_smoothed.mot' does not exist.
	Thrown at DelimFileAdapter.h:316 in extendRead().

In [6]:
# %% OpenSim pipeline.
if True:
    openSimPipelineDir = os.path.join('./', "opensimPipeline")        
    
    if True:
        openSimFolderName = 'OpenSimData'
    else:
        openSimFolderName = os.path.join('OpenSimData', 
                                            poseDetector + suff_pd)
        if not markerDataFolderNameSuffix is None:
            openSimFolderName = os.path.join(openSimFolderName,
                                                markerDataFolderNameSuffix)
    
    openSimDir = os.path.join('./', openSimFolderName)        
    outputScaledModelDir = os.path.join(openSimDir, 'Model')

    # # Check if shoulder model.
    # if 'shoulder' in sessionMetadata['openSimModel']:
    #     suffix_model = '_shoulder'
    # else:
    #     suffix_model = ''
    
    # Scaling.    
    if True:
        os.makedirs(outputScaledModelDir, exist_ok=True)
        # Path setup file.
        #CHANGEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
        genericSetupFile4ScalingName = (
            'Setup_scaling_RajagopalModified2016_withArms_KA.xml')
        pathGenericSetupFile4Scaling = os.path.join(
            openSimPipelineDir, 'Scaling', genericSetupFile4ScalingName)
        # Path model file.
        pathGenericModel4Scaling = os.path.join(
            openSimPipelineDir, 'Models', 
            sessionMetadata['openSimModel'] + '.osim')            
        # Path TRC file.
        ##CHANGEEEEEEEEEEEEEEEEEEEEEEEEEEEE
        pathTRCFile4Scaling =  '~/projects/Hand_Detection/trace_files/m002/'
        # Get time range.
        try:
            thresholdPosition = 0.003
            maxThreshold = 0.015
            increment = 0.001
            success = False
            while thresholdPosition <= maxThreshold and not success:
                try:
                    timeRange4Scaling = getScaleTimeRange(
                        pathTRCFile4Scaling,
                        thresholdPosition=thresholdPosition,
                        thresholdTime=0.1, removeRoot=True)
                    success = True
                except Exception as e:
                    print(f"Attempt with thresholdPosition {thresholdPosition} failed: {e}")
                    thresholdPosition += increment  # Increase the threshold for the next iteration

            # Run scale tool.
            print('Running Scaling')
            pathScaledModel = runScaleTool(
                pathGenericSetupFile4Scaling, pathGenericModel4Scaling,
                sessionMetadata['mass_kg'], pathTRCFile4Scaling, 
                timeRange4Scaling, outputScaledModelDir,
                subjectHeight=sessionMetadata['height_m'], 
                suffix_model='')
        except Exception as e:
            if len(e.args) == 2: # specific exception
                raise Exception(e.args[0], e.args[1])
            elif len(e.args) == 1: # generic exception
                exception = "Musculoskeletal model scaling failed. Verify your setup and try again. Visit https://www.opencap.ai/best-pratices to learn more about data collection and https://www.opencap.ai/troubleshooting for potential causes for a failed neutral pose."
                raise Exception(exception, traceback.format_exc())
        # Extract one frame from videos to verify neutral pose.
        staticImagesFolderDir = os.path.join('./', 
                                                'NeutralPoseImages')
        os.makedirs(staticImagesFolderDir, exist_ok=True)
        # popNeutralPoseImages(cameraDirectories, cameras2Use, 
        #                         timeRange4Scaling[0], staticImagesFolderDir,
        #                         trial_id, writeVideo = True)   
        pathOutputIK = pathScaledModel[:-5]+'.mot'     
    
    # Inverse kinematics.
    if True:
        outputIKDir = os.path.join(openSimDir, 'Kinematics')
        os.makedirs(outputIKDir, exist_ok=True)
        # Check if there is a scaled model.
        pathScaledModel = os.path.join(outputScaledModelDir, 
                                        sessionMetadata['openSimModel'] + 
                                        "_scaled.osim")
        if os.path.exists(pathScaledModel):
            # Path setup file.
            genericSetupFile4IKName = 'Setup_IK{}.xml'.format('')
            pathGenericSetupFile4IK = os.path.join(
                openSimPipelineDir, 'IK', genericSetupFile4IKName)
            # Path TRC file.
            #CHANGE
            pathTRCFile4IK = pathAugmentedOutputFiles[trialName]
            # Run IK tool. 
            print('Running Inverse Kinematics')
            try:
                pathOutputIK = runIKTool(
                    pathGenericSetupFile4IK, pathScaledModel, 
                    pathTRCFile4IK, outputIKDir)
            except Exception as e:
                if len(e.args) == 2: # specific exception
                    raise Exception(e.args[0], e.args[1])
                elif len(e.args) == 1: # generic exception
                    exception = "Inverse kinematics failed. Verify your setup and try again. Visit https://www.opencap.ai/best-pratices to learn more about data collection and https://www.opencap.ai/troubleshooting for potential causes for a failed trial."
                    raise Exception(exception, traceback.format_exc())
        else:
            raise ValueError("No scaled model available.")
    
    # Write body transforms to json for visualization.
    outputJsonVisDir = os.path.join(sessionDir,'VisualizerJsons',
                                    trialName)
    os.makedirs(outputJsonVisDir,exist_ok=True)
    outputJsonVisPath = os.path.join(outputJsonVisDir,
                                        trialName + '.json')
    generateVisualizerJson(pathScaledModel, pathOutputIK,
                            outputJsonVisPath, 
                            vertical_offset=vertical_offset)  
    

NameError: name 'sessionMetadata' is not defined